# House Prices using SpeedML

1. Setting up
2. Data Exploration & Outlier Detection & Fix
3. Feature Engineering
4. Model building & Hyperparameter Tuning
5. Model Evaluation & Prediction

## 1. Setting up

In [161]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [162]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [163]:
train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [164]:
test_id = test.Id
test.drop('Id', axis=1, inplace=True)
train.drop('Id', axis=1, inplace=True)

## 2. Data Exploration & Outlier Detection & Fix 

In [165]:
def show_missing_values(df):
    df_nan = df.isnull().sum()[df.isnull().sum()>0].sort_values(ascending=False)
    df_nan_percent = np.round(df_nan*100/df.shape[0],2)
    print(pd.concat([df_nan, df_nan_percent], axis = 1, keys = ['#Nan', '%Nan']))

In [166]:
train_tmp = train.drop('SalePrice',axis=1)
total = pd.concat([train_tmp, test], axis=0).reset_index(drop=True)

In [167]:
total.columns

Index([u'MSSubClass', u'MSZoning', u'LotFrontage', u'LotArea', u'Street',
       u'Alley', u'LotShape', u'LandContour', u'Utilities', u'LotConfig',
       u'LandSlope', u'Neighborhood', u'Condition1', u'Condition2',
       u'BldgType', u'HouseStyle', u'OverallQual', u'OverallCond',
       u'YearBuilt', u'YearRemodAdd', u'RoofStyle', u'RoofMatl',
       u'Exterior1st', u'Exterior2nd', u'MasVnrType', u'MasVnrArea',
       u'ExterQual', u'ExterCond', u'Foundation', u'BsmtQual', u'BsmtCond',
       u'BsmtExposure', u'BsmtFinType1', u'BsmtFinSF1', u'BsmtFinType2',
       u'BsmtFinSF2', u'BsmtUnfSF', u'TotalBsmtSF', u'Heating', u'HeatingQC',
       u'CentralAir', u'Electrical', u'1stFlrSF', u'2ndFlrSF', u'LowQualFinSF',
       u'GrLivArea', u'BsmtFullBath', u'BsmtHalfBath', u'FullBath',
       u'HalfBath', u'BedroomAbvGr', u'KitchenAbvGr', u'KitchenQual',
       u'TotRmsAbvGrd', u'Functional', u'Fireplaces', u'FireplaceQu',
       u'GarageType', u'GarageYrBlt', u'GarageFinish', u'GarageCars'

Let's explore PoolQC

In [168]:
total.PoolArea.value_counts()

0      2906
561       1
555       1
519       1
800       1
738       1
648       1
576       1
512       1
480       1
444       1
368       1
228       1
144       1
Name: PoolArea, dtype: int64

In [169]:
total.PoolQC.value_counts()

Ex    4
Gd    4
Fa    2
Name: PoolQC, dtype: int64

Looking at PoolArea of zero it shows that the missing values of Pool Quality are actually cases where the house doesn't have a pool

In [170]:
total.PoolQC.fillna('Na', inplace=True)

In [171]:
show_missing_values(total)

              #Nan   %Nan
MiscFeature   2814  96.40
Alley         2721  93.22
Fence         2348  80.44
FireplaceQu   1420  48.65
LotFrontage    486  16.65
GarageFinish   159   5.45
GarageYrBlt    159   5.45
GarageQual     159   5.45
GarageCond     159   5.45
GarageType     157   5.38
BsmtExposure    82   2.81
BsmtCond        82   2.81
BsmtQual        81   2.77
BsmtFinType2    80   2.74
BsmtFinType1    79   2.71
MasVnrType      24   0.82
MasVnrArea      23   0.79
MSZoning         4   0.14
BsmtFullBath     2   0.07
BsmtHalfBath     2   0.07
Functional       2   0.07
Utilities        2   0.07
Exterior2nd      1   0.03
Exterior1st      1   0.03
SaleType         1   0.03
BsmtFinSF1       1   0.03
BsmtFinSF2       1   0.03
BsmtUnfSF        1   0.03
Electrical       1   0.03
KitchenQual      1   0.03
GarageCars       1   0.03
GarageArea       1   0.03
TotalBsmtSF      1   0.03


Let's check for the other missing values

In [172]:
total.MiscFeature.value_counts()

Shed    95
Gar2     5
Othr     4
TenC     1
Name: MiscFeature, dtype: int64

In [173]:
total.Alley.value_counts()

Grvl    120
Pave     78
Name: Alley, dtype: int64

total_missing = total.isnull().sum()[total.isnull().sum()>0].sort_values(ascending=False)
for features in total:
    if total[features].isnull().sum()>0 and total[features].dtypes == 'object':
        print(total[features].value_counts())
        features_for_NA.append(features)

It can be seen that all these are cases of not applicable (NA). Let's fill the Nan's with NA then.

In [174]:
print(features_for_NA)

['MiscFeature', 'Fence', 'Alley', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'Fence', 'MiscFeature', 'MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'KitchenQual', 'Functional', 'SaleType']


Exclude some features that have to be handled by imputing with mode

In [175]:
features_to_exclude = ['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd',
                       'Electrical', 'KitchenQual', 'Functional', 'SaleType']

In [176]:
for ftr in features_for_NA:
    if ftr not in features_to_exclude:
        total[ftr].fillna('NA', inplace=True)

In [177]:
show_missing_values(total)

              #Nan   %Nan
LotFrontage    486  16.65
GarageYrBlt    159   5.45
MasVnrArea      23   0.79
MSZoning         4   0.14
Functional       2   0.07
Utilities        2   0.07
BsmtHalfBath     2   0.07
BsmtFullBath     2   0.07
BsmtFinSF1       1   0.03
Exterior1st      1   0.03
Exterior2nd      1   0.03
SaleType         1   0.03
BsmtFinSF2       1   0.03
BsmtUnfSF        1   0.03
GarageArea       1   0.03
Electrical       1   0.03
KitchenQual      1   0.03
GarageCars       1   0.03
TotalBsmtSF      1   0.03


Of these, the object missing values are to be filled using their mode

In [178]:
total_missing = total.isnull().sum()[total.isnull().sum()>0].sort_values(ascending=False)
for features in total:
    if total[features].isnull().sum()>0 and total[features].dtypes == 'object':
        total[features].fillna(total[features].mode()[0], inplace=True)

In [179]:
show_missing_values(total)

              #Nan   %Nan
LotFrontage    486  16.65
GarageYrBlt    159   5.45
MasVnrArea      23   0.79
BsmtHalfBath     2   0.07
BsmtFullBath     2   0.07
GarageArea       1   0.03
GarageCars       1   0.03
TotalBsmtSF      1   0.03
BsmtUnfSF        1   0.03
BsmtFinSF2       1   0.03
BsmtFinSF1       1   0.03


Apart from LotFrontage, GarageYrBlt and MasVnrArea, the other nan's can safely be filled with 0

In [180]:
features_to_exclude = ['LotFrontage', 'GarageYrBlt', 'MasVnrArea']
for features in total:
    if (total[features].isnull().sum()>0) and (features not in features_to_exclude):
        total[features].fillna(0, inplace=True)

In [181]:
show_missing_values(total)

             #Nan   %Nan
LotFrontage   486  16.65
GarageYrBlt   159   5.45
MasVnrArea     23   0.79
